# Coordinate Transformations

## About coordinate transformations

There are a number of coordinate transformations specific to neutron-scattering such as conversion from time-of-flight to wavelength.
Such coordinate transformations are also referred to as "unit conversion", but here we avoid this terminology, to avoid confusion with [conversion of array units using `sc.to_unit`](https://scipp.github.io/generated/functions/scipp.to_unit.html#scipp.to_unit).

Scipp provides [coordinate transformations using `sc.transform_coords`](https://scipp.github.io/user-guide/coordinate-transformations.html).
Scippneutron defines concrete transformations for time-of-flight neutron scattering, as well als building blocks for customizing transformations.
Both are discussed in the following.

## Built-in transformations

Built-in transformations are used by the `convert` function, which is provided for convenience.
Here we describe the more direct approach.
The transformations typically require two components:

1. A definition of the beamline geometry which defines have, e.g., scattering angles are to be computed from positions of beamline components, such as sample and detector positions.
2. A definition of the scattering dynamics, e.g., how wavelength can be computed from time-of-flight for an elastic scattering process.

Pre-defined standard components can be imported from `scippneutron.tof.conversions`:

In [ ]:
import scipp as sc
from scippneutron.tof.conversions import beamline, elastic

Here `beamline` defines a "simple" time-of-flight beamline with source-, sample-, and detector positions:

In [ ]:
sc.show_graph(beamline(scatter=True))

For a given input coordinate, say, `'tof'`, `elastic` defined how, e.g., `wavlength` or `dspacing` may be computed:

In [ ]:
sc.show_graph(elastic('tof'))

The two transformation graphs depicted above can now be used to transform raw data to, e.g., `wavelength`:

In [ ]:
import scippneutron as scn

da = scn.load(scn.data.get_path('PG3_4844_event.nxs'))
da.transform_coords('wavelength',
                    graph={
                        **beamline(scatter=True),
                        **elastic('tof')
                    })

Note how `transform_coords` automatically handles event data.

We may use `beamline(scatter=False)` for conversions without scattering process, e.g., for processing neutron monitors, or for imaging beamlines.

## Customizing transformations

Many neutron beamlines are more complex than what is assumed by the simple built-in transformation graphs.
The coordinate transformation mechanism is completely generic and is customizable.
We provide examples in the following.

### Diffraction calibration

In [ ]:
def dspacing(tof, tzero, difc):
    difc = sc.reciprocal(difc)
    return difc * (tof - tzero)


graph = {'dspacing': dspacing}
sc.show_graph(graph)

In [ ]:
from mantid.simpleapi import Load, LoadDiffCal

#ws = LoadDiffCal('/home/simon/data/PG3_FERNS_d4832_2011_08_24.cal', InstrumentName='PG3', WorkspaceName='ws')
#ws = LoadDiffCal('/home/simon/data/TrainingCourseData/PG3_golden.cal', InstrumentName='PG3', WorkspaceName='ws')
ws = Load('PG3_4844_event.nxs')
ws = LoadDiffCal('/home/simon/data/TrainingCourseData/PG3_golden.cal', InputWorkspace='ws', WorkspaceName='ws')

In [ ]:
cal = scn.from_mantid(ws[0])
tzero = cal['tzero'].data.copy()
tzero = tzero.rename_dims({'row':'spectrum'})
tzero.unit = 'us'
difc = cal['difc'].data.copy()
difc = difc.rename_dims({'row':'spectrum'})
difc.unit = 'us/angstrom'

In [ ]:
import scippneutron as scn

da = scn.load(scn.data.get_path('PG3_4844_event.nxs'))
da.coords['tzero'] = tzero
da.coords['difc'] = difc
da.transform_coords('dspacing', graph=graph)

### Gravity correction

For techniques such as SANS probing low Q regions the basic conversion approach may not be sufficient.
A computation of $2\theta$ may need to take into account gravity since the gravity drop after scattering is not negligible.
This can be achieved by replacing the function for computation of `two_theta` in built in graph.
We define a custom `two_theta` function:

In [ ]:
from scipp.constants import m_n, h, g


def two_theta(gravity, wavelength, incident_beam, scattered_beam):
    L2 = sc.norm(scattered_beam)
    # Arbitrary internal convention: beam=z, gravity=y
    y = sc.dot(scattered_beam, gravity) / sc.norm(gravity)
    n = sc.cross(incident_beam, gravity)
    n /= sc.norm(n)
    x = sc.dot(scattered_beam, n)
    wavelength = sc.to_unit(wavelength, 'm', copy=False)
    drop = g * m_n**2 / (2 * h**2) * wavelength**2 * L2**2
    return sc.asin(sc.sqrt(x**2 + (y + drop)**2) / L2)

This can be used to setup a modified graph for the coordinate transformation:

In [ ]:
from scippneutron.tof.conversions import beamline, elastic

q_with_gravity = {**beamline(scatter=True), **elastic('tof')}
q_with_gravity['two_theta'] = two_theta

The result is as follows:

In [ ]:
del q_with_gravity['energy']  # not necessary, clarifies conversion graph
del q_with_gravity['dspacing']  # not necessary, clarifies conversion graph
sc.show_graph(q_with_gravity, simplified=True)

We can use this to convert SANS data to $Q$.
Our custom transformation graph requires a `gravity` input coordinate, so we define one.
In this case (LARMOR beamline) "up" is along the `y` axis:

In [ ]:
da = scn.data.tutorial_dense_data()
# Convert to bin centers so we can later bin into Q bins
da.coords['tof'] = 0.5 * (da.coords['tof']['tof', :-1] +
                          da.coords['tof']['tof', 1:])
da.coords['gravity'] = sc.vector(unit='m/s**2', value=[0, -9.81, 0])
da_gravity = da.transform_coords('Q', graph=q_with_gravity)
da_gravity

As a finaly step we may then bin our data into $Q$ bins:

In [ ]:
q_bins = sc.linspace(dim='Q', unit='1/angstrom', start=0.0, stop=15.0, num=100)
da_gravity = sc.bin(da_gravity.flatten(to='Q'), edges=[q_bins]).bins.sum()
da_gravity.plot(norm='log')

### 2-D Rietveld

In [ ]:
def dspacing_perpendicular(wavelength, two_theta):
    return sc.sqrt(wavelength * wavelength - 2 * sc.Unit('angstrom*angstrom') *
                   sc.log(sc.cos(0.5 * two_theta)))

In [ ]:
d_perp = '$d_\perp$'
from scippneutron.tof.conversions import beamline, elastic

graph = {**beamline(scatter=True), **elastic('tof'), **beamline(scatter=True)}
graph['d'] = 'dspacing'
graph[d_perp] = dspacing_perpendicular
#sc.show_graph(graph)

In [ ]:
da2 = da.transform_coords(['d', d_perp], graph=graph)

sc.bin(da2.events,
       edges=[
           sc.linspace(dim='d', unit='angstrom', start=0.0, stop=2.0, num=10),
           sc.linspace(dim=d_perp, unit='angstrom', start=0.0, stop=2.0, num=10)
       ]).plot()
#sc.bin(da2, edges=[d, d_perp], erase=['spectrum', 'tof']).plot()